In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
import sys
sys.path.append('../src')

from math_utils import *
from model import *

In [2]:
# Generate random dataset
np.random.seed(42)
x_train = np.random.randn(1000, 3)

# Invent a label based on the three input features
y_train = x_train[:, 0] * 2 + x_train[:, 1] * -1.5 + x_train[:, 2] * 0.5

In [3]:
x_test = np.random.randn(2000, 3)
y_test = x_test[:, 0] * 2 + x_test[:, 1] * -1.5 + x_test[:, 2] * 0.5

In [4]:
model = ELM(3, 200, init_method='normal')

model.fit(x_train, y_train.reshape(-1, 1))

In [5]:
# compute the prediction loss
y_pred = model.predict(x_test)
mse = np.mean((y_test - y_pred) ** 2)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.015241875727959672


In [6]:
# Calculate mean absolute error without sklearn
y_pred_test = model.predict(x_test)
mae = np.mean(np.abs(y_test - y_pred_test))
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 0.04717465591299475


test QR

In [7]:
m = 10
n = 5
X = np.random.randn(m, n)
X

array([[-2.19750101,  1.26871389,  0.19022322, -0.3796245 ,  0.85715839],
       [ 1.5246386 , -2.80191157,  0.43426365, -1.42664921,  0.49911051],
       [ 0.1279272 ,  0.63572653, -0.49924761, -1.87324011, -0.45036819],
       [ 0.40684764,  1.01539233, -1.1718313 ,  1.02706043, -0.64787229],
       [ 0.36244988, -0.50989395,  0.18217063,  0.89187763, -1.22701408],
       [-0.6799059 ,  0.18496259,  2.40158366, -0.86318971,  2.04013389],
       [ 2.0044233 , -0.09731115, -0.29815253, -0.6920652 , -0.06820253],
       [-1.40671943, -0.07623902,  1.49688027, -0.61108506, -0.2407243 ],
       [-1.70114047,  0.37406209,  0.26448168,  0.06370236, -0.21682705],
       [-0.29275792,  0.50190023, -0.0288166 ,  0.31497187,  0.21498282]])

In [8]:
householder_vectors, R = thin_QR(X)

In [9]:
householder_vectors

[array([-0.87591786,  0.21167183,  0.01776066,  0.05648433,  0.0503204 ,
        -0.09439412,  0.27828231, -0.19530063, -0.23617631, -0.04064478]),
 array([-0.9271335 ,  0.13301706,  0.23169833, -0.06250348, -0.02943307,
         0.17193922, -0.14814644, -0.0905092 ,  0.06760515]),
 array([-0.72622074, -0.13833384,  0.02937933,  0.6050674 ,  0.16195235,
         0.22518954, -0.09544872,  0.02116905]),
 array([ 0.78824455,  0.23214212, -0.42070043, -0.33935636, -0.13608281,
         0.11586292,  0.02619595]),
 array([-0.89705128,  0.02027502,  0.02327757, -0.41632821, -0.13235739,
         0.05914755])]

In [10]:
R

array([[ 4.11159632e+00, -1.88420913e+00, -1.21805892e+00,
        -2.38514498e-01, -6.73177208e-01],
       [ 3.56824763e-17,  2.83664948e+00, -1.56067322e+00,
         9.16477764e-01, -5.17779463e-01],
       [ 1.19554094e-17, -2.78221576e-17,  2.47567260e+00,
        -9.82135320e-01,  1.61906261e+00],
       [ 1.88968360e-16, -3.50506849e-17, -7.81270262e-16,
        -2.71626593e+00,  5.75151703e-01],
       [ 5.10094558e-17, -1.20513093e-17,  2.31502841e-16,
        -3.90872591e-18,  1.93817354e+00]])

In [11]:
result = apply_householders_matrix(householder_vectors, np.vstack([R, np.zeros((m - n, n))]))
result

array([[-2.19750101,  1.26871389,  0.19022322, -0.3796245 ,  0.85715839],
       [ 1.5246386 , -2.80191157,  0.43426365, -1.42664921,  0.49911051],
       [ 0.1279272 ,  0.63572653, -0.49924761, -1.87324011, -0.45036819],
       [ 0.40684764,  1.01539233, -1.1718313 ,  1.02706043, -0.64787229],
       [ 0.36244988, -0.50989395,  0.18217063,  0.89187763, -1.22701408],
       [-0.6799059 ,  0.18496259,  2.40158366, -0.86318971,  2.04013389],
       [ 2.0044233 , -0.09731115, -0.29815253, -0.6920652 , -0.06820253],
       [-1.40671943, -0.07623902,  1.49688027, -0.61108506, -0.2407243 ],
       [-1.70114047,  0.37406209,  0.26448168,  0.06370236, -0.21682705],
       [-0.29275792,  0.50190023, -0.0288166 ,  0.31497187,  0.21498282]])

In [12]:
X

array([[-2.19750101,  1.26871389,  0.19022322, -0.3796245 ,  0.85715839],
       [ 1.5246386 , -2.80191157,  0.43426365, -1.42664921,  0.49911051],
       [ 0.1279272 ,  0.63572653, -0.49924761, -1.87324011, -0.45036819],
       [ 0.40684764,  1.01539233, -1.1718313 ,  1.02706043, -0.64787229],
       [ 0.36244988, -0.50989395,  0.18217063,  0.89187763, -1.22701408],
       [-0.6799059 ,  0.18496259,  2.40158366, -0.86318971,  2.04013389],
       [ 2.0044233 , -0.09731115, -0.29815253, -0.6920652 , -0.06820253],
       [-1.40671943, -0.07623902,  1.49688027, -0.61108506, -0.2407243 ],
       [-1.70114047,  0.37406209,  0.26448168,  0.06370236, -0.21682705],
       [-0.29275792,  0.50190023, -0.0288166 ,  0.31497187,  0.21498282]])

test incremental qr

In [13]:
m = 10
n = 5
X = np.random.randn(m, n)
X

array([[ 1.12342729,  0.6769941 ,  0.19286184,  1.51842203,  0.42026853],
       [ 1.30496269,  0.60694261, -1.84191908,  1.30483476,  0.13517624],
       [-0.05195477,  0.28134835, -0.34498316,  0.22702886, -0.59187607],
       [ 1.27427226, -0.11358811, -0.38258384, -0.92492582,  0.26877579],
       [ 0.1871071 ,  0.38842461, -1.02725189, -1.6011812 ,  0.6255913 ],
       [ 1.48464186, -1.02635362,  1.58944128,  0.28391106, -0.0070629 ],
       [-0.3975487 , -1.38451802,  0.98772852, -0.77560636, -0.77980668],
       [ 0.99192631, -1.88298345, -1.52359781, -0.01813424,  0.29328443],
       [ 0.48473465, -0.30179849,  2.53036967, -0.22789256, -0.23078615],
       [-0.79698667,  0.12390682,  0.61496123,  1.20357805, -1.48928426]])

In [14]:
householder_vectors, R = thin_QR(X)
x = np.random.randn(m,1)
y = np.random.randn(m,1)
z = np.random.randn(m,1)

In [15]:
householder_vectors, R = incr_QR(x, householder_vectors, R)
householder_vectors, R = incr_QR(y, householder_vectors, R)
householder_vectors, R = incr_QR(z, householder_vectors, R)

In [16]:
result = apply_householders_matrix(householder_vectors, np.vstack([R, np.zeros((m-(n+3), (n+3)))]))
result

array([[ 1.12342729e+00,  6.76994097e-01,  1.92861843e-01,
         1.51842203e+00,  4.20268529e-01, -2.45995338e-01,
        -1.04692175e+00, -3.24619674e-01],
       [ 1.30496269e+00,  6.06942608e-01, -1.84191908e+00,
         1.30483476e+00,  1.35176241e-01,  4.46703611e-01,
         6.77969971e-01,  2.37731728e-01],
       [-5.19547709e-02,  2.81348347e-01, -3.44983162e-01,
         2.27028860e-01, -5.91876066e-01,  5.87057535e-01,
        -1.28472777e+00,  1.03078222e-01],
       [ 1.27427226e+00, -1.13588114e-01, -3.82583843e-01,
        -9.24925816e-01,  2.68775789e-01,  6.54865780e-03,
        -3.31024334e-01,  2.39800190e-02],
       [ 1.87107100e-01,  3.88424614e-01, -1.02725189e+00,
        -1.60118120e+00,  6.25591303e-01,  5.42423970e-01,
         4.54704396e-01, -7.50626822e-01],
       [ 1.48464186e+00, -1.02635362e+00,  1.58944128e+00,
         2.83911059e-01, -7.06290455e-03, -8.08963893e-01,
        -6.61659345e-02, -2.99549411e-01],
       [-3.97548697e-01, -1.384518

In [ ]:
# concat X, x, y, z
X = np.hstack([X, x, y, z])
X

array([[ 1.12342729e+00,  6.76994097e-01,  1.92861843e-01,
         1.51842203e+00,  4.20268529e-01, -2.45995338e-01,
        -1.04692175e+00, -3.24619674e-01],
       [ 1.30496269e+00,  6.06942608e-01, -1.84191908e+00,
         1.30483476e+00,  1.35176241e-01,  4.46703611e-01,
         6.77969971e-01,  2.37731728e-01],
       [-5.19547709e-02,  2.81348347e-01, -3.44983162e-01,
         2.27028860e-01, -5.91876066e-01,  5.87057535e-01,
        -1.28472777e+00,  1.03078222e-01],
       [ 1.27427226e+00, -1.13588114e-01, -3.82583843e-01,
        -9.24925816e-01,  2.68775789e-01,  6.54865780e-03,
        -3.31024334e-01,  2.39800190e-02],
       [ 1.87107100e-01,  3.88424614e-01, -1.02725189e+00,
        -1.60118120e+00,  6.25591303e-01,  5.42423970e-01,
         4.54704396e-01, -7.50626822e-01],
       [ 1.48464186e+00, -1.02635362e+00,  1.58944128e+00,
         2.83911059e-01, -7.06290455e-03, -8.08963893e-01,
        -6.61659345e-02, -2.99549411e-01],
       [-3.97548697e-01, -1.384518

In [18]:
np.equal(result, X)

array([[ True, False, False,  True, False, False, False, False],
       [ True, False, False, False, False, False, False, False],
       [False, False, False,  True,  True, False, False, False],
       [ True, False, False, False, False, False, False, False],
       [ True, False, False, False, False,  True,  True, False],
       [ True, False, False, False, False, False, False, False],
       [ True, False, False, False, False, False, False, False],
       [False, False,  True, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [ True, False,  True, False,  True, False, False, False]])

test dimensions

In [ ]:
X = np.random.randn(1000, 10)
Y = np.random.randn(1000, 50)
Z = np.random.randn(1000, 500)

In [24]:
householder_vectors, R = thin_QR(X)

In [25]:
householder_vectors, R = thin_QR(Y)

In [26]:
householder_vectors, R = thin_QR(Z)